## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

# Importing the dataset

In [ ]:
dataset=pd.read_csv('Churn_Modelling.csv')
# removing the rowmnumber,CustomerId,Surname column because they dont contibute to the end result.
X=dataset.iloc[:,3:-1].values
y=dataset.iloc[:,-1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


# Encoding categorical data

In [ ]:
#Label Encoding the gender column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])


# One hot Encoding the Geography column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Spliting the dataset into train set and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Feature Scaling

In [ ]:
# In ann and cnn all the column have to be featured scaled even if they contain only 1 or 0

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

#Building the ANN

## Intializing the ANN

In [ ]:
ann=tf.keras.models.Sequential()

## Create a input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

## Creating the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

## Creating the output layer

In [ ]:
#Only one unit because we need only 0 or 1 in the output
#We use sigmoid activation function for the output layer because its gives the prediction along with the probabilty of it happening
#For non-binary classification we should use the activation function softmax 
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Compiling the ANN

In [ ]:
#adam optimizer performs Stochastic Gradient descent
#For a binary classification we can always use binary_crossentropy as the loss function and for a categorical classification we can use categorical_crossentropy
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Training the ANN on the training set

In [ ]:
ann.fit(X_train,y_train,batch_size,epochs=100)

NameError: ignored

## Predicting for a single value

In [ ]:
# This gives the probabilty(Since we used the sigmoid function as activating function for output layer)
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.02311113]]


In [ ]:
# We assume values below probabilty of 0.5 would be false and above 0.5 would be true
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


## Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix¶

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1523   72]
 [ 194  211]]


0.867